In [1]:
#Main Function
import requests
from bs4 import BeautifulSoup
import time
import threading
import urllib.request #Save Image
#File
from datetime import datetime
import os
from PIL import ImageTk
import PIL.Image #네임스페이스 충돌
#UI
from tkinter import *
from tkinter import ttk
import tkinter.messagebox

In [2]:
# 더럽게 짠 부분 보고 욕은 하지 말아주세요 ㅎ,, 최대한 리팩토링 해오겠습니다.
# 부족한 부분에 대한 코드 리뷰 부탁드립니다 :D 
# 방식 -> 공식 홈페이지 랭크 HTML에서 닉네임, 인기도, 길드, 이미지 추출. 먼저 인기도부터 비교하여 조건에 맞지 않으면 다음 캐릭터, 조건에 맞으면 유니온, 업적 비교. 
# 모든 조건에 맞는 캐릭터는 suspect_info_list에 캐릭터 정보가 들어가며 이 결과를 토대로 파일 저장 및 출력 

class ChangeUiText():
    
    def ResetProgressText():
        label_progress["state"] = 'normal'
        label_suspect_amount["state"] = 'normal'
        label_expect_time["state"] = 'normal' 
        label_before_result.config(text ='')
        ChangeUiText.ProgressPercent(0)
        ChangeUiText.SuspectAmount(0)
        ChangeUiText.ExpectedTime(0)
                
    def ActivateButton(change_text):
        button_activate.config(text = change_text)
    
    def ProgressPercent(num):
        label_progress.config(text = ( str(num)+'%' ))
        
    def SuspectAmount(amount):
        label_suspect_amount.config(text = '['+ str(amount)+ '명]')
        
    def ExpectedTime(time):
        if time <= 0:
            label_expect_time.config(text = '')
        else:
            label_expect_time.config(text = '예상 시간 : '+str(round(time,2))+ '초')
            
    def BeforeResultLabelMessage():
        label_progress["state"] = 'disable'
        label_suspect_amount["state"] = 'disable'
        label_expect_time["state"] = 'disable'  
        label_before_result.config(text ='결과를 저장하는 중 입니다....')
            
class File():
    
    def MakeDirectory(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    
    def SetupFile(file_route):
        file = open(file_route,'w', encoding = "utf-8")
        file.close()
    
    def WriteTextToFile(file_route, text):
        file = open(file_route,'a', encoding = "utf-8")
        file.write(text+'\n')
        file.close()
    
def ActivateMainFunction_Thread():
    # Call work function
    t1=threading.Thread(target=ActivateMainFunction)
    t1.setDaemon(True)
    t1.deamon = True
    t1.start()

def DeActivateMainFunction():
    global g_is_running
    g_is_running = False
    ChangeUiText.ActivateButton(change_text = 'Activate')
    ChangeUiText.ResetProgressText()
    return 0

def ActivateMainFunction():
    #If Click DeActivateBtn
    global g_is_running
    if g_is_running == True :
        DeActivateMainFunction()
        tkinter.messagebox.showinfo("Message", "사용자 요청에 따라 작동을 중지합니다.")
        return 0
    else : 
        g_is_running = True
        ChangeUiText.ActivateButton(change_text = 'DeActivate')
    
    #Checking Error
    exist_error = CheckError()
    if exist_error == True:
        DeActivateMainFunction()
        tkinter.messagebox.showerror("Error Message", "오류를 발견하여 작동을 중지합니다.\n수정 후 재시도 바랍니다.")
        return 0    
    
    server_code = SetServerCode()
    job_code = SetJobCode()
    page_min = int(entry_page_min.get())
    page_max = int(entry_page_max.get())
    global g_union_range_min, g_union_range_max, g_achive_range_min, g_achive_range_max 
    g_union_range_min = int(entry_union_min.get())
    g_union_range_max = int(entry_union_max.get())
    g_achive_range_min = int(entry_achieve_min.get())
    g_achive_range_max = int(entry_achieve_max.get())
    if is_checked_ingido_btn.get() == False:
        ingido_range_min = int(entry_ingido_min.get())
        ingido_range_max = int(entry_ingido_max.get())
    suspect_info_list = []
    suspect_amount = 0
    total_progress = (page_max-page_min+1)*10
    #Ui Set
    ChangeUiText.ProgressPercent(0)
    ChangeUiText.SuspectAmount(0)
    
    for present_page in range(page_min, page_max+1):
        start_time = time.time()
        #extract nickname 
        maple_url = SetMapleRankURL(str(present_page), server_code, job_code)
        maple_html = ReturnHTML(maple_url)
        nick_info_list = SetNickInfo(maple_html)   
        for i in range(0,10):
            if g_is_running == False: #If User click Deactivate button when already running, end function
                DeActivateMainFunction()
                return 0

            if is_checked_ingido_btn.get() == False:
                if int(nick_info_list[i][2]) >= ingido_range_min and int(nick_info_list[i][2]) <= ingido_range_max:
                    find_suspect_info = InvestigateCharacter(nick_info_list[i][0])
                else:
                    time.sleep(0.1)
                    present_progress = (present_page-page_min)*10+(i+1)
                    ChangeUiText.ProgressPercent(round((present_progress/total_progress)*100,1))                    
                    continue
            else:
                find_suspect_info = InvestigateCharacter(nick_info_list[i][0])
         
            if find_suspect_info != 'Not Suspect':
                suspect_amount += 1
                ChangeUiText.SuspectAmount(suspect_amount)
                suspect_info_list.append(find_suspect_info)
                suspect_info_list[suspect_amount-1].append(nick_info_list[i][1]) #level
                suspect_info_list[suspect_amount-1].append(nick_info_list[i][2]) #ingido
                suspect_info_list[suspect_amount-1].append(nick_info_list[i][3]) #guild
                suspect_info_list[suspect_amount-1].append(nick_info_list[i][4]) #character img url  
            #Print progress percentage
            present_progress = (present_page-page_min)*10+(i+1)
            ChangeUiText.ProgressPercent(round((present_progress/total_progress)*100,1))
        #Print expected complete time
        end_time = time.time()
        ChangeUiText.ExpectedTime((end_time - start_time) * (page_max - present_page))
        
    #Make Result File    
    if suspect_info_list != []:
        ChangeUiText.BeforeResultLabelMessage()
        dir_name = datetime.today().strftime("%Y-%m-%d-%H%M%S") 
        File.MakeDirectory(dir_name) 
        File.SetupFile(dir_name+'/★결과.txt')
        SaveResultFile(suspect_info_list, dir_name)
        PrintResult(suspect_info_list, suspect_amount, dir_name)
    else:
        tkinter.messagebox.showinfo("Message", "조건에 일치하는 캐릭터가 없습니다.") 
    DeActivateMainFunction()
    
def CloseResult():
    try:
        result_window.destroy()
    except:
        pass
    
def SaveResultFile(suspect_info_list, dir_name):
    for suspect_info in suspect_info_list:
        nick = suspect_info[0]
        level = suspect_info[3]
        img_url = suspect_info[6]
        urllib.request.urlretrieve(img_url, './'+dir_name+'/'
                                   +nick
                                   +'.jpg')
        File.WriteTextToFile(file_route = dir_name+'/★결과.txt', 
                             text = nick+'('+ level+')'
                                   +'\n[유니온] ' + str(suspect_info[1])
                                   +'\n[업적] ' + str(suspect_info[2])
                                   +'\n[인기도] '  + str(suspect_info[4])
                                   +'\n[길드] ' + suspect_info[5]
                                   +'\n')

def PrintResult(suspect_info_list, suspect_amount, dir_name):
    tkinter.messagebox.showinfo("Message", "결과가 '"+dir_name+"' 폴더에 저장되었습니다.") 
    def BeforeInfo():
#         if g_is_running == True:
#             tkinter.messagebox.showinfo("Message", "메인 기능이 활성화되어 기존 결과창이 종료됩니다.") 
#             result_window.destroy()
#             return 0
        global character_page
        if character_page <= 0: #First Page -> Last Page
            character_page = suspect_amount - 1 
            ShowCharacterInfo()
        else:
            character_page -= 1
            ShowCharacterInfo()

    def NextInfo():
#         if g_is_running == True:
#             tkinter.messagebox.showinfo("Message", "메인 기능이 활성화되어 기존 결과창이 종료됩니다.") 
#             result_window.destroy()
#             return 0        
        global character_page
        if character_page+1 >= suspect_amount: #Last Page -> First Page
            character_page = 0
            ShowCharacterInfo()
        else:
            character_page += 1
            ShowCharacterInfo()

    def ShowCharacterInfo():
        global character_img, character_page
        nick = suspect_info_list[character_page][0]
        level = suspect_info_list[character_page][3]
        union = str(suspect_info_list[character_page][1])
        achieve = str(suspect_info_list[character_page][2])
        ingido = str(suspect_info_list[character_page][4])
        guild = suspect_info_list[character_page][5]
        
        result_window.title("결과["+str(character_page+1)+'/'+str(suspect_amount)+']')
        character_img = ImageTk.PhotoImage(PIL.Image.open('./'+dir_name+'/'+nick+'.jpg'))
        label_character_img.config(image = character_img)
        label_nick.config(text = nick+'('+ level+')')
        label_union.config(text = '유니온 : '+union)
        label_achieve.config(text = '업적 : '+achieve)
        label_ingido.config(text = '인기도 : '+ingido)
        label_guild.config(text = '길드 : '+guild)      
    global character_page  
    character_page = 0
    result_window = Toplevel()
    result_window.title("결과")
    result_window.geometry('230x318')
    result_window.option_add('*Font', '맑은고딕 9')
    result_window.resizable(False, False)
    label_character_img = Label(result_window)
    label_nick = Label(result_window)
    label_union = Label(result_window)
    label_achieve = Label(result_window)
    label_ingido = Label(result_window)
    label_guild = Label(result_window)    
    BeforeBtn = Button(result_window, width = 10, text = '이전', command = BeforeInfo)
    NextBtn = Button(result_window, width = 10, text = '다음', command = NextInfo)

    label_character_img.pack()
    label_nick.pack()
    label_union.pack()
    label_achieve.pack()
    label_ingido.pack()
    label_guild.pack()
    BeforeBtn.place(x = 5, y = 288)
    NextBtn.place(x = 145, y = 288)
    ShowCharacterInfo()     
        
def CheckError():
    exist_error = False
    error_message = ""
    error_num = 0
    
    if combobox_server.get() == '서버 선택':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 서버를 선택해주세요.\n"
        
    if combobox_job_category.get() == '직업군 선택':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 직업군을 선택해주세요.\n"

    if combobox_job.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 직업을 선택해주세요.\n"
        
    if entry_union_min.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 유니온 최소 범위를 입력해주세요.\n"
        
    if entry_union_max.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 유니온 최대 범위를 입력해주세요.\n"
        
    try:
        if int(entry_union_min.get()) > int(entry_union_max.get()):
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 유니온 최대 수치가 최소보다 작습니다.\n"  
    except:
        if entry_union_min.get() != '' or entry_union_max.get() != '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 유니온 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n"        
        
    if entry_achieve_min.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 업적 최소 범위를 입력해주세요.\n"
        
    if entry_achieve_max.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 업적 최대 범위를 입력해주세요.\n"      
        
    try:
        if int(entry_achieve_min.get()) > int(entry_achieve_max.get()):
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 업적 최대 수치가 최소보다 작습니다.\n"  
    except:
        if entry_achieve_min.get() != '' or entry_achieve_max.get() != '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 업적 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n" 
    
    if is_checked_ingido_btn.get() == False:
        
        if entry_ingido_min.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 인기도 최소 범위를 입력해주세요.\n"

        if entry_ingido_max.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 인기도 최대 범위를 입력해주세요.\n"     
            
        try:
            if int(entry_ingido_min.get()) > int(entry_ingido_max.get()):
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 인기도 최대 수치가 최소보다 작습니다.\n"  
        except:
            if entry_ingido_min.get() != '' or entry_ingido_max.get() != '':
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 인기도 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n"          
        
    if entry_page_min.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 페이지 최소 범위를 입력해주세요.\n" 
        
    if entry_page_max.get() == '':
        exist_error = True
        error_num += 1
        error_message += str(error_num)+". 페이지 최대 범위를 입력해주세요.\n" 
        
    try:
        if int(entry_page_min.get()) > int(entry_page_max.get()):
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 페이지 최대 수치가 최소보다 작습니다.\n"  
    except:
        if entry_page_min.get() != '' or entry_page_max.get() != '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 페이지 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n"      
        
    if exist_error == True:
        tkinter.messagebox.showerror("Error Message", "오류를 발견하였습니다!\n\n" + error_message)   
        return True
    else: 
        return False

def SetServerCode():
    server = combobox_server.get()
    if server == '전체월드': server_code = ''
    elif server == '리부트2': server_code = '&w=1'
    elif server == '리부트1': server_code = '&w=2'
    elif server == '오로라': server_code = '&w=3'
    elif server == '레드': server_code = '&w=4'
    elif server == '이노시스': server_code = '&w=5'
    elif server == '유니온': server_code = '&w=6'
    elif server == '스카니아': server_code = '&w=7'
    elif server == '루나': server_code = '&w=8'
    elif server == '제니스': server_code = '&w=9'
    elif server == '크로아': server_code = '&w=10'
    elif server == '베라': server_code = '&w=11'
    elif server == '엘리시움': server_code = '&w=12'
    elif server == '아케인': server_code = '&w=13'
    elif server == '노바': server_code = '&w=14'

    return server_code

def SetJobCode():
    job = combobox_job.get()
    if job == '히어로' : job_code = '&j=1&d=12'
    elif job == '팔라딘' : job_code = '&j=1&d=22'
    elif job == '다크나이트' : job_code = '&j=1&d=32'
        
    elif job == '불독' : job_code = '&j=2&d=12'
    elif job == '썬콜' : job_code = '&j=2&d=22'
    elif job == '비숍' : job_code = '&j=2&d=32'
        
    elif job == '보우마스터' : job_code = '&j=3&d=12'
    elif job == '신궁' : job_code = '&j=3&d=22'
    elif job == '패스파인더' : job_code = '&j=3&d=32'
        
    elif job == '나이트로드' : job_code = '&j=4&d=12'
    elif job == '섀도어' : job_code = '&j=4&d=22'
    elif job == '듀얼블레이더' : job_code = '&j=4&d=34'
        
    elif job == '바이퍼' : job_code = '&j=5&d=12'
    elif job == '캡틴' : job_code = '&j=5&d=22'
    elif job == '캐논슈터' : job_code = '&j=5&d=32'
        
    elif job == '소울마스터' : job_code = '&j=6&d=11'
    elif job == '플레임위자드' : job_code = '&j=6&d=12'
    elif job == '윈드브레이커' : job_code = '&j=6&d=13'
    elif job == '나이트워커' : job_code = '&j=6&d=14'    
    elif job == '스트라이커' : job_code = '&j=6&d=15'  
    elif job == '미하일' : job_code = '&j=6&d=16'  

    elif job == '배틀메이지' : job_code = '&j=10&d=32'
    elif job == '와일드헌터' : job_code = '&j=10&d=33'
    elif job == '메카닉' : job_code = '&j=10&d=35'
    elif job == '데몬슬레이어' : job_code = '&j=10&d=31'
    elif job == '데몬어벤져' : job_code = '&j=10&d=131'
    elif job == '제논' : job_code = '&j=10&d=36'
    elif job == '블래스터' : job_code = '&j=10&d=37'
        
    elif job == '아란' : job_code = '&j=7'
    elif job == '에반' : job_code = '&j=8'
    elif job == '메르세데스' : job_code = '&j=11'  
    elif job == '팬텀' : job_code = '&j=12'
    elif job == '루미너스' : job_code = '&j=13'
    elif job == '은월' : job_code = '&j=18'
        
    elif job == '카이저' : job_code = '&j=14'
    elif job == '엔버' : job_code = '&j=15'
    elif job == '카데나' : job_code = '&j=21'
    elif job == '카인' : job_code = '&j=26'

    elif job == '아크' : job_code = '&j=23'
    elif job == '일리움' : job_code = '&j=22'
    elif job == '아델' : job_code = '&j=25'

    elif job == '호영' : job_code = '&j=24'
    elif job == '라라' : job_code = '&j=28'

    elif job == '키네시스' : job_code = '&j=20'
    elif job == '제로' : job_code = '&j=17' 

    return job_code
    
def SetMapleRankURL(page, job_code, server_code):
    return "https://maplestory.nexon.com/Ranking/World/Total?page="+page+job_code+server_code
    
def ReturnHTML(url):
    #Set User-Agent(Chrome)
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content
    return BeautifulSoup(html, "html.parser")   

def SetNickInfo(maple_html):
    nick_info_list = []
    info_html_list = maple_html.select('table.rank_table > tbody > tr')
    for info in info_html_list:
        nick = info.select_one('td.left > dl > dt').get_text()
        level = info.contents[5].get_text()
        ingido = info.contents[9].get_text().replace(',','')
        guild = info.contents[11].get_text()
        if guild == '' : guild = '-'
        char_img_url = info.select_one('span.char_img > img').get('src')
        nick_info_list.append([nick, level, ingido, guild, char_img_url])
    return nick_info_list
        
def SearchUnion(nick):
    try:
        url = "https://maplestory.nexon.com/Ranking/Union?c="+nick
        html = ReturnHTML(url)
        union_score = html.select_one("tr.search_com_chk").contents[5].get_text().replace(',','')
        return int(union_score)
    except:
        #If doen't have Union Score
        return 0

def SearchAchievement(nick):
    try:
        url = "https://maplestory.nexon.com/Ranking/Achievement?c="+nick
        html = ReturnHTML(url)
        achieve_score = html.select_one("tr.search_com_chk").contents[7].get_text().replace(',','')
        return int(achieve_score)
    except:
        #If doen't set representative Character
        return 0    
    
def InvestigateCharacter(nick):
    global g_union_range_min, g_union_range_max, g_achive_range_min, g_achive_range_max 
    union_score = SearchUnion(nick)
    
    if union_score >= g_union_range_min and union_score <= g_union_range_max:
        achieve_score = SearchAchievement(nick)
        #업적 0점도 포함 기능을 활성화했다면,
        if is_checked_no_achieve_btn.get() == True:
            if (achieve_score >= g_achive_range_min and achieve_score <= g_achive_range_max) or achieve_score == 0:
                if achieve_score == 0 : achieve_score = '-' #업적 0점은 '-' 텍스트로 출력
                return [nick, union_score, achieve_score]
            else:
                return 'Not Suspect'
        #아닌 경우 조건
        else:
            if achieve_score >= g_achive_range_min and achieve_score <= g_achive_range_max:
                return [nick, union_score, achieve_score]
            else:
                return 'Not Suspect'
    else:
        return 'Not Suspect' 
    
#Global Var
g_is_running = False
g_union_range_min = 0
g_union_range_max = 0
g_achive_range_min = 0
g_achive_range_max = 0

In [3]:
def ShowJobList(event):
    job_category = combobox_job_category.get()
    combobox_job.set('')
    
    if job_category == '[모전]' : job_list = ['히어로', '팔라딘', '다크나이트']
    elif job_category == '[모법]' : job_list = ['불독', '썬콜', '비숍']
    elif job_category == '[모궁]' : job_list = ['보우마스터', '신궁', '패스파인더']
    elif job_category == '[모도]' : job_list = ['나이트로드', '섀도어', '듀얼블레이더']
    elif job_category == '[모해]' : job_list = ['바이퍼', '캡틴', '캐논슈터']    
    elif job_category == '[시그너스]' : job_list = ['소울마스터', '플레임위자드', '윈드브레이커', '나이트워커', '스트라이커', '미하일'] 
    elif job_category == '[레지스탕스]' : job_list = ['배틀메이지', '와일드헌터', '메카닉','데몬슬레이어', '데몬어벤져', '제논', '블래스터'] 
    elif job_category == '[영웅]': job_list = ['아란', '에반', '메르세데스', '팬텀', '루미너스', '은월'] 
    elif job_category == '[노바]': job_list = ['카이저', '엔버', '카데나', '카인'] 
    elif job_category == '[레프]': job_list = ['아크', '일리움', '아델'] 
    elif job_category == '[아니마]': job_list = ['호영', '라라']     
    elif job_category == '[혼밥족]': job_list = ['키네시스', '제로'] 
    
    combobox_job["state"] = "readonly"
    combobox_job["values"] = job_list
    
def EnabledIngido():
    if is_checked_ingido_btn.get() == True:
        tkinter.messagebox.showinfo("Message", "기능이 활성화되어 인기도 상관 없이 캐릭터를 검색합니다\n(검색 속도가 대폭 저하됩니다)") 
        entry_ingido_min.delete(0, len(entry_ingido_min.get()))
        entry_ingido_max.delete(0, len(entry_ingido_max.get()))
        entry_ingido_min['state'] = 'disabled'
        entry_ingido_max['state'] = 'disabled'      
    else:
        entry_ingido_min['state'] = 'normal'
        entry_ingido_max['state'] = 'normal'   
        
def AchieveCheckBtnMessage():
    if is_checked_no_achieve_btn.get() == True:
        tkinter.messagebox.showinfo("Message", "기능이 활성화되어 업적이 없는 캐릭터도 검색합니다.") 

#[Set Windows]
win = Tk()
win.title('추노')    
win.geometry('430x330')
win.option_add('*Font', '맑은고딕 10')
win.resizable(False, False)

#[Set Image]
label_image = Label(win)
main_img_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
main_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_img_url, stream=True).raw))
label_image.config(image = main_img)
label_image.pack()

#[Set Server]
label_server = Label(win, text = '[서버 설정]')

server_list =['전체월드', '루나', '스카니아', '엘리시움', '크로아', '베라', 
              '오로라', '레드', '제니스', '이노시스', '유니온', 
              '아케인', '노바', '리부트1', '리부트2'] 

combobox_server = ttk.Combobox(win, 
                               width = 10, 
                               values = server_list, 
                               state="readonly")
combobox_server.set("서버 선택")

label_server.place(x = 100 , y = 100)
combobox_server.place(x = 200, y = 100)

#[Set Job]
label_job = Label(win, text = '[직업 설정]')

job_category =['[모전]', '[모법]', '[모궁]', '[모도]', '[모해]',
               '[시그너스]',
               '[영웅]',
               '[레지스탕스]',
               '[노바]',
               '[레프]',
               '[아니마]',
               '[혼밥족]'
               ] 
combobox_job_category = ttk.Combobox(win, 
                                     width = 10, 
                                     values = job_category, 
                                     state="readonly")
    #job_category Choice Event
combobox_job_category.bind("<<ComboboxSelected>>", ShowJobList)
    #job_category Text
combobox_job_category.set("직업군 선택")

combobox_job = ttk.Combobox(win, width = 10, state="disabled")

label_job.place(x = 100 , y = 130)
combobox_job_category.place(x = 200, y = 130)    
combobox_job.place(x = 300, y = 130)    

#[Set Union Score]
label_union = Label(win, text = '[유니온 범위]')

entry_union_min = Entry(win, width = 5)
label_union_range = Label(win, text = '-')
entry_union_max = Entry(win, width = 5)

label_union.place(x = 93, y = 160)
entry_union_min.place(x = 200, y = 160)
entry_union_max.place(x = 260, y = 160)
label_union_range.place(x = 240, y = 160)

#[Set Achievement Score]
label_achieve = Label(win, text = '[업적 범위]')

entry_achieve_min = Entry(win, width = 5)
label_achieve_range = Label(win, text = '-')
entry_achieve_max = Entry(win, width = 5)

label_achieve.place(x = 100, y = 190)
entry_achieve_min.place(x = 200, y = 190)
entry_achieve_max.place(x = 260, y = 190)
label_achieve_range.place(x = 240, y = 190)  

#[Set Ingido]
label_ingido = Label(win, text = '[인기도 범위]')

entry_ingido_min = Entry(win, width = 5)
label_ingido_range = Label(win, text = '-')
entry_ingido_max = Entry(win, width = 5)

label_ingido.place(x = 93, y = 220)
entry_ingido_min.place(x = 200, y = 220)
entry_ingido_max.place(x = 260, y = 220)
label_ingido_range.place(x = 240, y = 220)

#[Set page]
label_page = Label(win, text = '[페이지 범위]')

entry_page_min = Entry(win, width = 5)
label_page_range = Label(win, text = '-')
entry_page_max = Entry(win, width = 5)

label_page.place(x = 93, y = 250)
entry_page_min.place(x = 200, y = 250)
entry_page_max.place(x = 260, y = 250)
label_page_range.place(x = 240, y = 250)

#[Progress Label]
label_progress = Label(win)
label_progress.place(x = 195 , y = 273)

label_suspect_amount = Label(win)
label_suspect_amount.place(x = 235 , y = 273)

label_expect_time = Label(win)
label_expect_time.place(x = 290 , y = 273)

label_before_result = Label(win)
label_before_result.place(x = 130, y = 273)

#[Option Check Button]
is_checked_ingido_btn = BooleanVar()
ingido_chbtn = Checkbutton(win,
                    text = "인기도 기능 OFF",
                    variable = is_checked_ingido_btn,
                    font = '맑은고딕 8')
ingido_chbtn.config(command = EnabledIngido)
ingido_chbtn.place(x = 290, y = 290)  # 0.35 0.9

is_checked_no_achieve_btn = BooleanVar()
achieve_chbtn = Checkbutton(win,
                    text = "업적이 없어도 검색",
                    variable = is_checked_no_achieve_btn,
                    font = '맑은고딕 8')
achieve_chbtn.config(command = AchieveCheckBtnMessage)
achieve_chbtn.place(x = 290, y = 310)  # 0.35 0.9

#[Activate Button]
button_activate = Button(win, 
                         width = 20, 
                         text = 'Activate',
                         command = ActivateMainFunction_Thread)
button_activate.place(x = 120 , y = 300)

#Start GUI
win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\JaGwon\AppData\Local\Temp/ipykernel_3648/3817829289.py", line 185, in BeforeInfo
    ShowCharacterInfo()
  File "C:\Users\JaGwon\AppData\Local\Temp/ipykernel_3648/3817829289.py", line 202, in ShowCharacterInfo
    nick = suspect_info_list[character_page][0]
IndexError: list index out of range
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\JaGwon\AppData\Local\Temp/ipykernel_3648/3817829289.py", line 185, in BeforeInfo
    ShowCharacterInfo()
  File "C:\Users\JaGwon\AppData\Local\Temp/ipykernel_3648/3817829289.py", line 202, in ShowCharacterInfo
    nick = suspect_info_list[character_page][0]
IndexError: list index out of range
